In [ ]:
from src.all_in_one import *

In [ ]:
# Screening
scr = get_df(scr_path, scr_sheet)
unique_scr = scr.drop_duplicates(subset='rid')

In [ ]:
td_dataset = Dataset(config_file, td_sheet)
td = get_df(td_path, td_sheet)
td = pd.merge(td, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
filtered_td = filter_dataframe_on_date(td, td_dataset.dvars, start, end, FilterType.IN)

In [ ]:
td

In [ ]:
filtered_td

In [ ]:
# filtered_td.to_clipboard(index=False)

In [ ]:
cws_dataset = Dataset(config_file, cws_sheet)
cws = get_df(cws_path, cws_sheet)

In [ ]:
cws

In [ ]:
td_cws = pd.merge(td, cws[['rid', 'cws_s1', 'cws_s2']], on='rid', how='left')